# Рекомендательные системы

## Урок 4. Рекомендательные системы на основе контента

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируютсяpip install ipykernel

In [8]:
pip install implicit


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: C:\Users\Азм\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip



  Using cached implicit-0.6.2-cp38-cp38-win_amd64.whl (647 kB)


In [9]:
import pandas as pd
import numpy as np
import os, sys

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from src.utils import prefilter_items
from src.recommenders import MainRecommender

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

data = pd.read_csv('../../../raw_data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

item_features = pd.read_csv('../../../raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features=item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

model = AlternatingLeastSquares(factors=20,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recommender = MainRecommender(data_train)

C:\Users\Азм\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Азм\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Decreased # items from 86865 to 5001


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5001/5001 [00:00<00:00, 35463.58it/s]


In [10]:
user = 2375
print(recommender.get_similar_items_recommendation(user, N=6))

[1029743, 1106523, 5569230, 916122, 1044078, 844179]


In [11]:
user = 1
print(recommender.get_similar_items_recommendation(user, N=5))

[1029743, 1106523, 5569230, 916122, 1044078]


In [12]:
user = 999
print(recommender.get_similar_items_recommendation(user, N=8))

[1029743, 1106523, 5569230, 916122, 1044078, 844179, 1126899, 1070820]
